In [1]:
import warnings

warnings.filterwarnings("ignore", message="Pydantic serializer warnings")

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain_ollama import ChatOllama

model = ChatOllama(model="llama3.1:8b", temperature=0)

In [4]:
from pydantic import BaseModel, Field

class ColourContext(BaseModel):
    favourite_colour: str = Field(
        default="blue", 
        description="The user's preferred colour used for UI elements."
    )
    least_favourite_colour: str = "yellow"

In [5]:
from langchain.agents import create_agent

agent = create_agent(
    model=model,
    context_schema=ColourContext,
)

In [6]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [HumanMessage(content="What is my favourite colour?")]},
    context=ColourContext()
)

In [7]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='What is my favourite colour?', additional_kwargs={}, response_metadata={}, id='4600c79b-247f-40f6-b0f5-fb4c820f67b8'),
              AIMessage(content="I don't have any information about your personal preferences, including your favorite color. Would you like to share it with me?", additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2026-01-07T00:22:33.299691353Z', 'done': True, 'done_reason': 'stop', 'total_duration': 8593848139, 'load_duration': 131988281, 'prompt_eval_count': 16, 'prompt_eval_duration': 1303241065, 'eval_count': 26, 'eval_duration': 7109623666, 'logprobs': None, 'model_name': 'llama3.1:8b', 'model_provider': 'ollama'}, id='lc_run--019b95d5-54bf-7ec3-b38f-6ce40ae7a300-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 16, 'output_tokens': 26, 'total_tokens': 42})]}


In [8]:
print(response["messages"][-1].content)

I don't have any information about your personal preferences, including your favorite color. Would you like to share it with me?


## Accessing Context

In [9]:
from langchain.tools import tool, ToolRuntime

@tool
def get_favourite_colour(runtime: ToolRuntime) -> str:
    """Get the favourite colour of the user"""
    return runtime.context.favourite_colour

@tool
def get_least_favourite_colour(runtime: ToolRuntime) -> str:
    """Get the least favourite colour of the user"""
    return runtime.context.least_favourite_colour

In [10]:
agent = create_agent(
    model=model,
    tools=[get_favourite_colour, get_least_favourite_colour],
    context_schema=ColourContext,
)

In [11]:
response = agent.invoke(
    {"messages": [HumanMessage(content="What is my favourite colour?")]},
    context=ColourContext() 
)

pprint(response)

{'messages': [HumanMessage(content='What is my favourite colour?', additional_kwargs={}, response_metadata={}, id='de0bf153-8d81-4975-a3ed-cd267b16b02c'),
              AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2026-01-07T00:22:59.777893265Z', 'done': True, 'done_reason': 'stop', 'total_duration': 26394813466, 'load_duration': 143852181, 'prompt_eval_count': 188, 'prompt_eval_duration': 20990986122, 'eval_count': 15, 'eval_duration': 5221349725, 'logprobs': None, 'model_name': 'llama3.1:8b', 'model_provider': 'ollama'}, id='lc_run--019b95d5-76a5-7920-b7d8-b4cd73049619-0', tool_calls=[{'name': 'get_favourite_colour', 'args': {}, 'id': '6a595245-10b5-4074-8402-d22ad7979581', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 188, 'output_tokens': 15, 'total_tokens': 203}),
              ToolMessage(content='blue', name='get_favourite_colour', id='e85a278e-f623-4f96-a6d3-bcc0f6a1e3de', tool_call_id='6a5

In [12]:
print(response["messages"][-1].content)

Based on the output of the `get_favourite_colour` function, your favourite colour is blue.


In [13]:
response = agent.invoke(
    {"messages": [HumanMessage(content="What is my least favourite colour?")]},
    context=ColourContext() 
)

pprint(response)

{'messages': [HumanMessage(content='What is my least favourite colour?', additional_kwargs={}, response_metadata={}, id='4594b8ec-505f-4ab9-a34a-9e93a6f11bcb'),
              AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2026-01-07T00:23:34.19732791Z', 'done': True, 'done_reason': 'stop', 'total_duration': 21944136398, 'load_duration': 157065119, 'prompt_eval_count': 189, 'prompt_eval_duration': 16854269685, 'eval_count': 16, 'eval_duration': 4894004735, 'logprobs': None, 'model_name': 'llama3.1:8b', 'model_provider': 'ollama'}, id='lc_run--019b95d6-0e7c-75f3-bf71-2549b857c9d2-0', tool_calls=[{'name': 'get_least_favourite_colour', 'args': {}, 'id': '25366106-ea08-4018-9709-3cf1cd38187d', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 189, 'output_tokens': 16, 'total_tokens': 205}),
              ToolMessage(content='yellow', name='get_least_favourite_colour', id='9ee83ad9-bfbe-4e9f-8163-8b03ab3337d0'

In [14]:
print(response["messages"][-1].content)

Based on the output of the `get_least_favourite_colour` function, it appears that your least favourite colour is yellow.


In [15]:
response = agent.invoke(
    {"messages": [HumanMessage(content="What is my favourite colour?")]},
    context=ColourContext(favourite_colour="green")
)

pprint(response)

{'messages': [HumanMessage(content='What is my favourite colour?', additional_kwargs={}, response_metadata={}, id='8f0f2141-77b0-42d7-923c-14de91e7ff26'),
              AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2026-01-07T00:24:07.954893219Z', 'done': True, 'done_reason': 'stop', 'total_duration': 21223058618, 'load_duration': 156014640, 'prompt_eval_count': 188, 'prompt_eval_duration': 16768396975, 'eval_count': 15, 'eval_duration': 4265181794, 'logprobs': None, 'model_name': 'llama3.1:8b', 'model_provider': 'ollama'}, id='lc_run--019b95d6-952a-7583-b2b2-f3481d9f7ebd-0', tool_calls=[{'name': 'get_favourite_colour', 'args': {}, 'id': '0fec49af-f392-4b69-ac57-8052173ed812', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 188, 'output_tokens': 15, 'total_tokens': 203}),
              ToolMessage(content='green', name='get_favourite_colour', id='9b38f7eb-b6fe-4c93-bdb0-16effa67fd7f', tool_call_id='0f

In [16]:
print(response["messages"][-1].content)

Based on the output of the `get_favourite_colour` function, it appears that your favourite colour is green.
